In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [2]:
## Load dataset
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [4]:
## Preprocessing
data = data.drop(['RowNumber','CustomerId','Surname'], axis=1)
data.head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


In [5]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [6]:
# One-hot encode Geography
one_hot_encoder = OneHotEncoder()
geo_encoded = one_hot_encoder.fit_transform(data[['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoded.toarray(), columns=one_hot_encoder.get_feature_names_out(['Geography']))

# Replace original Geography column
data = pd.concat([data.drop('Geography', axis=1).reset_index(drop=True), geo_encoded_df], axis=1)
data.head(4)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0


In [7]:
with open("label_encoder_gender.pkl", "wb") as f:
    pickle.dump(label_encoder_gender, f)

# === Save OneHotEncoder for Geography ===
with open("one_hot_encoder_geography.pkl", "wb") as f:
    pickle.dump(one_hot_encoder, f)

In [8]:
# Independent features (X) – all columns except 'Exited'
X = data.drop('Exited', axis=1)

# Dependent feature (y) – only the target column
y = data['Exited']

# Optional: preview shapes
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Shape of X: (10000, 12)
Shape of y: (10000,)


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (8000, 12)
X_test shape: (2000, 12)


In [10]:
# Initialize StandardScaler
scaler = StandardScaler()

# Fit on X_train and transform both sets
X_train = scaler.fit_transform(X_train)
X_test= scaler.transform(X_test)

In [11]:
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

In [12]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [13]:
import  tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [14]:
model = Sequential([
    Dense(64, activation='relu', input_shape = (X_train.shape[1],)), ## HL1 connected with input layer
    Dense(32,activation = 'relu'),## HL2
    Dense(1, activation ='sigmoid') ## OPL
])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
import tensorflow
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.01)

In [17]:
# Compilation
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [18]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%M%d-%H%M%S")
tensorflow_callbacks = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [19]:
early_stoping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [20]:
history = model.fit(
    X_train, y_train, validation_data = (X_test, y_test), epochs = 30,
    callbacks = [tensorflow_callbacks,early_stoping_callback]
)

Epoch 1/30


250/250 [==============================] - 5s 9ms/step - loss: 0.4001 - accuracy: 0.8321 - val_loss: 0.3479 - val_accuracy: 0.8580
Epoch 2/30
250/250 [==============================] - 2s 7ms/step - loss: 0.3542 - accuracy: 0.8525 - val_loss: 0.3543 - val_accuracy: 0.8640
Epoch 3/30
250/250 [==============================] - 2s 7ms/step - loss: 0.3479 - accuracy: 0.8566 - val_loss: 0.3482 - val_accuracy: 0.8625
Epoch 4/30
250/250 [==============================] - 1s 5ms/step - loss: 0.3447 - accuracy: 0.8587 - val_loss: 0.3570 - val_accuracy: 0.8630
Epoch 5/30
250/250 [==============================] - 1s 5ms/step - loss: 0.3402 - accuracy: 0.8562 - val_loss: 0.3481 - val_accuracy: 0.8625
Epoch 6/30
250/250 [==============================] - 1s 5ms/step - loss: 0.3393 - accuracy: 0.8610 - val_loss: 0.3443 - val_accuracy: 0.8590
Epoch 7/30
250/250 [==============================] - 1s 5ms/step - loss: 0.3363 - accuracy: 0.8641 - val_loss: 0.3520 - val_accuracy: 0.8625
Epoc

In [21]:
model.save('model.h5')

c:\Users\mohsi\OneDrive\Desktop\deepLearning\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [22]:
%load_ext tensorboard

In [23]:
%tensorboard --logdir logs/fit --port=6007

Reusing TensorBoard on port 6007 (pid 12192), started 22:38:21 ago. (Use '!kill 12192' to kill it.)